# Create a retreat mask from basins and Calfin

Make sure all data sets have been downloaded prior to run this notetook

In [ ]:
import geopandas as gp
gp.options.io_engine = "pyogrio"
import pandas as pd
from tqdm.auto import tqdm
import xarray as xr
from geocube.api.core import make_geocube
import geocube
import numpy as np
from joblib import Parallel, delayed
from pathlib import Path
from typing import Union

## Prepare the IMBIE basins. Load, make valid geometry

In [ ]:
fronts_path = Path("../data/fronts")
fronts_path.mkdir(parents=True, exist_ok=True)
intermediate_path = Path("../data/intermediate")
intermediate_path.mkdir(parents=True, exist_ok=True)
basins_path = Path("../data/basins/")
calfin_path = Path("../data/calfin")

In [ ]:
imbie = gp.read_file(basins_path / Path("GRE_Basins_IMBIE2_v1.3.shp")).to_crs("EPSG:3413")
valid_geometry = imbie.make_valid()
imbie.geometry = valid_geometry
imbie = imbie[imbie["SUBREGION1"] != "ICE_CAP"].dissolve()
valid_geometry = imbie.make_valid()
imbie.geometry = valid_geometry
imbie_gp = gp.GeoDataFrame(geometry=imbie.geometry, crs="EPSG:3413")
imbie_gp.to_file(basins_path / Path("imbie.gpkg"))

In [ ]:
shelves = gp.read_file(basins_path / Path("GRE_Basins_shelf_extensions.gpkg")).to_crs("EPSG:3413").dissolve()
valid_geometry = shelves.make_valid()
shelves.geometry = valid_geometry
imbie = imbie.union(shelves)
imbie_gp = gp.GeoDataFrame(geometry=imbie.geometry, crs="EPSG:3413")
imbie_gp.to_file(basins_path / Path("imbie_shelves.gpkg"))

## Prepare the IMBIE basins. Load, make valid, and dissolve

In [ ]:
calfin = gp.read_file(calfin_path / Path("termini_1972-2019_Greenland_closed_v1.0.shp")).to_crs("EPSG:3413")
valid_geometry = calfin.make_valid()
calfin.geometry = valid_geometry

## Merge dissolved IMBIE with closed, dissolved Calfin polygons

This creates the outline from which we are going to subtract front positions

In [ ]:
calfin_dissolved = calfin.dissolve()
calfin_dissolved.to_file(intermediate_path / Path("calfin_dissolved.gpkg"))

In [ ]:
union = imbie.union(calfin_dissolved)
union_gp = gp.GeoDataFrame(geometry=union, crs="EPSG:3413").dissolve()
union_gp.to_file(intermediate_path / Path("union.gpkg"))

In [ ]:
# Create the geometry / bounding box
# This was taken from
# https://gis.stackexchange.com/questions/436243/how-to-properly-use-geocube-geom-parameter-with-crs-other-than-4326
bbox = (-653000.0, -3384350., 879700.0,  -632750.0)
geom = geocube.geo_utils.geobox.mapping(geocube.geo_utils.geobox.box(*bbox))
geom["crs"] = {"properties": {"name": "EPSG:3413"}}

In [ ]:
def create_ds(ds1, ds2, date, geom, crs: str = "EPSG:3413", grid: float = 150, out_path: Path = Path("../data/fronts")):
    
    front = ds1.dissolve().buffer(5)
    front.to_file(out_path / Path(f"g{grid}m_terminus_{date.year}-{date.month}.gpkg"))
    diff = ds2.difference(front)
    n = len(diff)
    diff_df = {"land_ice_area_fraction_retreat": np.ones(n)}
    diff_gp = gp.GeoDataFrame(data=diff_df, geometry=diff, crs=crs)
    diff_gp.to_file(out_path / Path(f"g{grid}m_frontretreat_{date.year}-{date.month}.gpkg"))
    ds = make_geocube(vector_data=diff_gp, geom=geom, resolution=[grid, grid]).fillna(0)
    ds = ds.expand_dims("time")
    ds["time"] = ("time", [pd.to_datetime(f"{date.year}-{date.month}-01")], 
            {
            "_FillValue": False,
            "units": "days since 1972-01-01",
            "calendar": "gregorian_proleptic",
            "axis": "T",
            "long_name": "time",
            "unlimited": True
        },
    )
    ds["time_bounds"] = (["time", "bounds"], [[pd.to_datetime(f"{date.year}-{date.month}-01") , 
                                               pd.to_datetime(f"{date.year}-{date.month+1}-01")]], 
          {  "_FillValue": False, "coordinates": False
        },
    )
    ds["time"].attrs = {"bounds": "time_bounds"}
    ds["land_ice_area_fraction_retreat"].attrs = {"units": "1"}
    fn = out_path / Path(f"g{grid}m_frontretreat_{date.year}-{date.month}.nc")
    comp = dict(zlib=True, complevel=2)
    encoding = {var: comp for var in ds.data_vars if var != "time_bounds"}
    ds.time.encoding = {"units": "days since 1972-01-01"}
    del ds.time_bounds.attrs["coordinates"]
    ds.time_bounds.encoding = {"units": "days since 1972-01-01", "coordinates": None}
    ds.to_netcdf(fn, encoding=encoding)
    del ds
    return fn

In [ ]:
grid = 150

In [ ]:
calfin["Date"] = pd.to_datetime(calfin["Date"])
calfin_ds = calfin.set_index("Date").groupby(by=pd.Grouper(freq='ME'))
nt = len(calfin_ds)

crs: str = "EPSG:3413"

result = []
i = 0
for date, ds in tqdm(calfin_ds):
    if len(ds) > 0:
        if i > 0:
            new_geom = old_ds.dissolve().union(ds.dissolve())
            ds = gp.GeoDataFrame(geometry=new_geom, crs=crs)
        fn = create_ds(ds, imbie, date, geom, grid=grid, out_path=fronts_path)
        result.append(fn)
        old_ds = ds
        i += 1


In [ ]:
ds = xr.open_mfdataset(result, parallel=True,)
comp = dict(zlib=True, complevel=2)
encoding = {var: comp for var in ds.data_vars if var != "time_bounds"}
ds.time.encoding.update({"_FillValue": None})
ds.time_bounds.encoding.update({"_FillValue": None, "coordinates": None})
ds.to_netcdf(fronts_path / Path(f"pism_g{grid}m_frontretreat_calfin_1972_2019.nc"), encoding=encoding, unlimited_dims='time')